In [ ]:
import numpy as np
import math
import time
import torch
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
from Modules.training_functions import *
from Modules.pennylane_functions import *

# if gpu available, set device to gpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using the GPU")
else:
    device = torch.device("cpu")
    print("WARNING: Could not find GPU, using the CPU")
T=10
all=np.load(f'Data/dataset_ld_{ld_dim}_{9}.npy')
all=all[:200]
for i in range(9):
    x=np.load(f'Data/dataset_ld_{ld_dim}_{i}.npy')
    all=np.concatenate((x[:200],all))

mnist_images=all
np.random.shuffle(mnist_images)
mnist_images = torch.tensor(mnist_images).to(device)

# make dataloader
data_loader = torch.utils.data.DataLoader(mnist_images, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
qc_array=np.array([0,64,96,112,120,124])
min_array=np.array([0.1,0.05,0.01,0.08])
layer_array=np.array([5,10,20,50])
print(NUM_QUBITS)
print(T)
zero = torch.zeros(BATCH_SIZE, 2**NUM_QUBITS-ld_dim).to(device)
for layer_indx in range(len(layer_array)):
    n_layer=layer_array[layer_indx]
    for q_indx in range(len(qc_array)):
        qc=qc_array[q_indx]
        for min_indx in range(len(min_array)):
            min_b=min_array[min_indx]

            betas      = np.insert(np.linspace(10e-8,min_b, T), 0, 0)
            print(np.shape(betas))
            alphas     = 1 - betas
            alphas_bar = np.cumprod(alphas)
            pi         = math.pi
            betas      = torch.tensor(betas).float().to(device)
            alphas     = torch.tensor(alphas).float().to(device)
            alphas_bar = torch.tensor(alphas_bar).float().to(device)
            theta_1    = Variable(torch.rand((n_layer*3*NUM_QUBITS+n_layer*3*(NUM_QUBITS)), device = device), requires_grad=True)
            optimizer = torch.optim.Adam([theta_1], lr = LEARNING_RATE)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = SCHEDULER_PATIENCE, gamma = SCHEDULER_GAMMA, verbose = False)
            trained_thetas_1 = []
            loss_history = []
            best_loss = 1e10

            for epoch in range(NUM_EPOCHS):
                print(epoch)

                t0 = time.time()
                num_batch=0
                tot_loss=0

                for image_batch in data_loader:

                    # extract batch of random times and betas
                    t = torch.randint(0, T, size = (BATCH_SIZE, ), device=device)
                    betas_batch = betas[t].to(device)
                    alphas_batch=alphas_bar[t].to(device)

                    # assemble input at t add noise (t+1)
                    target_batch = assemble_input(image_batch, t, alphas_bar,ld_dim ,device)
                    input_batch  = noise_step(target_batch, t+1, betas,ld_dim, device)
                    target_batch = target_batch / torch.norm(target_batch, dim = 1).view(-1, 1)
                    input_batch  = input_batch / torch.norm(input_batch, dim = 1).view(-1, 1)
                    

                    # concatenate the two tensors along the second dimension
                    input_batch = torch.cat((input_batch, zero), dim=1)
                    target_batch = torch.cat((target_batch, zero), dim=1)
                    # Feed to circuit, compute the loss and update the weights
                    num_batch+=1
                    loss = loss_fn_aq(qc,theta_1,n_layer, input_batch, target_batch)
                    tot_loss+=loss.item()
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

                # append parameters and print loss
                trained_thetas_1.append(theta_1.cpu().clone().detach().numpy())

                loss_history.append(tot_loss/num_batch)
                if loss.item()< best_loss:
                    best_loss=loss.item()

                # implement learning rate scheduler
                scheduler.step()


            # print every epoch
                print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={((time.time()-t0)*(NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))/60:.2f} min{min_b} nl{n_layer} QC{qc}')
                #print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={(((NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))):.2f} min{min_b} nl{n_layer}')
                
            np.save(f'thetas_T{T}_nl{n_layer}_min{min_b}_qc{qc}_{Q_ANCILLA}_ld{ld_dim}.npy',trained_thetas_1)
            np.save(f'loss__T{T}_nl{n_layer}_min{min_b}_qc{qc}_ancilla{Q_ANCILLA}_ld{ld_dim}.npy',loss_history)

(2000, 8)
7
10
(11,)
0
T=10 Epoch: 1/50 - Loss: 0.7323 b_loss=0.7323 - T: 3.08s/epoch ,tempo_previto=246.25 min0.1 nl5 QC0
1
T=10 Epoch: 2/50 - Loss: 0.5632 b_loss=0.5632 - T: 3.18s/epoch ,tempo_previto=254.25 min0.1 nl5 QC0
2
T=10 Epoch: 3/50 - Loss: 0.4409 b_loss=0.4409 - T: 3.28s/epoch ,tempo_previto=261.94 min0.1 nl5 QC0
3
T=10 Epoch: 4/50 - Loss: 0.3329 b_loss=0.3329 - T: 3.35s/epoch ,tempo_previto=267.54 min0.1 nl5 QC0
4
T=10 Epoch: 5/50 - Loss: 0.3061 b_loss=0.3061 - T: 3.54s/epoch ,tempo_previto=282.71 min0.1 nl5 QC0
5
T=10 Epoch: 6/50 - Loss: 0.2635 b_loss=0.2635 - T: 3.45s/epoch ,tempo_previto=275.40 min0.1 nl5 QC0
6
T=10 Epoch: 7/50 - Loss: 0.2361 b_loss=0.2361 - T: 3.55s/epoch ,tempo_previto=283.71 min0.1 nl5 QC0
7
T=10 Epoch: 8/50 - Loss: 0.2258 b_loss=0.2258 - T: 3.31s/epoch ,tempo_previto=264.53 min0.1 nl5 QC0
8
T=10 Epoch: 9/50 - Loss: 0.1641 b_loss=0.1641 - T: 3.35s/epoch ,tempo_previto=267.10 min0.1 nl5 QC0
9
T=10 Epoch: 10/50 - Loss: 0.1763 b_loss=0.1641 - T: 3.46s/e

T=10 Epoch: 30/50 - Loss: 0.1001 b_loss=0.0874 - T: 3.65s/epoch ,tempo_previto=272.09 min0.05 nl5 QC0
30
T=10 Epoch: 31/50 - Loss: 0.0949 b_loss=0.0874 - T: 3.73s/epoch ,tempo_previto=277.45 min0.05 nl5 QC0
31
T=10 Epoch: 32/50 - Loss: 0.0922 b_loss=0.0874 - T: 3.75s/epoch ,tempo_previto=279.40 min0.05 nl5 QC0
32
T=10 Epoch: 33/50 - Loss: 0.0930 b_loss=0.0874 - T: 3.85s/epoch ,tempo_previto=286.98 min0.05 nl5 QC0
33
T=10 Epoch: 34/50 - Loss: 0.0999 b_loss=0.0874 - T: 3.87s/epoch ,tempo_previto=287.81 min0.05 nl5 QC0
34
T=10 Epoch: 35/50 - Loss: 0.0933 b_loss=0.0874 - T: 4.04s/epoch ,tempo_previto=300.44 min0.05 nl5 QC0
35
T=10 Epoch: 36/50 - Loss: 0.0889 b_loss=0.0874 - T: 3.98s/epoch ,tempo_previto=295.99 min0.05 nl5 QC0
36
T=10 Epoch: 37/50 - Loss: 0.0998 b_loss=0.0874 - T: 4.13s/epoch ,tempo_previto=307.35 min0.05 nl5 QC0
37
T=10 Epoch: 38/50 - Loss: 0.0936 b_loss=0.0874 - T: 4.02s/epoch ,tempo_previto=299.05 min0.05 nl5 QC0
38
T=10 Epoch: 39/50 - Loss: 0.0885 b_loss=0.0874 - T: 4.1

T=10 Epoch: 9/50 - Loss: 0.1774 b_loss=0.1696 - T: 4.39s/epoch ,tempo_previto=284.45 min0.08 nl5 QC0
9
T=10 Epoch: 10/50 - Loss: 0.1410 b_loss=0.1410 - T: 4.57s/epoch ,tempo_previto=296.47 min0.08 nl5 QC0
10
T=10 Epoch: 11/50 - Loss: 0.1408 b_loss=0.1408 - T: 4.54s/epoch ,tempo_previto=294.19 min0.08 nl5 QC0
11
T=10 Epoch: 12/50 - Loss: 0.1469 b_loss=0.1408 - T: 4.66s/epoch ,tempo_previto=301.87 min0.08 nl5 QC0
12
T=10 Epoch: 13/50 - Loss: 0.1247 b_loss=0.1247 - T: 4.68s/epoch ,tempo_previto=303.41 min0.08 nl5 QC0
13
T=10 Epoch: 14/50 - Loss: 0.1199 b_loss=0.1199 - T: 4.87s/epoch ,tempo_previto=315.48 min0.08 nl5 QC0
14
T=10 Epoch: 15/50 - Loss: 0.1110 b_loss=0.1110 - T: 4.77s/epoch ,tempo_previto=308.68 min0.08 nl5 QC0
15
T=10 Epoch: 16/50 - Loss: 0.1095 b_loss=0.1095 - T: 4.81s/epoch ,tempo_previto=311.57 min0.08 nl5 QC0
16
T=10 Epoch: 17/50 - Loss: 0.1191 b_loss=0.1095 - T: 5.02s/epoch ,tempo_previto=325.11 min0.08 nl5 QC0
17
T=10 Epoch: 18/50 - Loss: 0.1043 b_loss=0.1043 - T: 4.99s

T=10 Epoch: 38/50 - Loss: 0.1298 b_loss=0.1245 - T: 4.87s/epoch ,tempo_previto=382.46 min0.1 nl5 QC64
38
T=10 Epoch: 39/50 - Loss: 0.1222 b_loss=0.1222 - T: 5.23s/epoch ,tempo_previto=410.41 min0.1 nl5 QC64
39
T=10 Epoch: 40/50 - Loss: 0.1219 b_loss=0.1219 - T: 5.24s/epoch ,tempo_previto=411.10 min0.1 nl5 QC64
40
T=10 Epoch: 41/50 - Loss: 0.1289 b_loss=0.1219 - T: 5.32s/epoch ,tempo_previto=417.75 min0.1 nl5 QC64
41
T=10 Epoch: 42/50 - Loss: 0.1301 b_loss=0.1219 - T: 6.68s/epoch ,tempo_previto=524.17 min0.1 nl5 QC64
42
T=10 Epoch: 43/50 - Loss: 0.1409 b_loss=0.1219 - T: 3.54s/epoch ,tempo_previto=277.99 min0.1 nl5 QC64
43
T=10 Epoch: 44/50 - Loss: 0.1326 b_loss=0.1219 - T: 3.45s/epoch ,tempo_previto=270.89 min0.1 nl5 QC64
44
T=10 Epoch: 45/50 - Loss: 0.1258 b_loss=0.1219 - T: 3.15s/epoch ,tempo_previto=246.82 min0.1 nl5 QC64
45
T=10 Epoch: 46/50 - Loss: 0.1299 b_loss=0.1219 - T: 3.48s/epoch ,tempo_previto=272.57 min0.1 nl5 QC64
46
T=10 Epoch: 47/50 - Loss: 0.1219 b_loss=0.1219 - T: 3.4

T=10 Epoch: 16/50 - Loss: 0.0471 b_loss=0.0453 - T: 3.62s/epoch ,tempo_previto=249.37 min0.01 nl5 QC64
16
T=10 Epoch: 17/50 - Loss: 0.0423 b_loss=0.0423 - T: 3.64s/epoch ,tempo_previto=250.69 min0.01 nl5 QC64
17
T=10 Epoch: 18/50 - Loss: 0.0447 b_loss=0.0423 - T: 3.68s/epoch ,tempo_previto=253.23 min0.01 nl5 QC64
18
T=10 Epoch: 19/50 - Loss: 0.0404 b_loss=0.0404 - T: 3.73s/epoch ,tempo_previto=256.74 min0.01 nl5 QC64
19
T=10 Epoch: 20/50 - Loss: 0.0365 b_loss=0.0365 - T: 3.77s/epoch ,tempo_previto=259.43 min0.01 nl5 QC64
20
T=10 Epoch: 21/50 - Loss: 0.0412 b_loss=0.0365 - T: 3.87s/epoch ,tempo_previto=266.02 min0.01 nl5 QC64
21
T=10 Epoch: 22/50 - Loss: 0.0410 b_loss=0.0365 - T: 4.08s/epoch ,tempo_previto=280.57 min0.01 nl5 QC64
22
T=10 Epoch: 23/50 - Loss: 0.0388 b_loss=0.0365 - T: 4.12s/epoch ,tempo_previto=283.66 min0.01 nl5 QC64
23
T=10 Epoch: 24/50 - Loss: 0.0396 b_loss=0.0365 - T: 4.17s/epoch ,tempo_previto=286.88 min0.01 nl5 QC64
24
T=10 Epoch: 25/50 - Loss: 0.0402 b_loss=0.0365

...

T=10 Epoch: 44/50 - Loss: 0.0065 b_loss=0.0058 - T: 15.43s/epoch ,tempo_previto=425.92 min0.01 nl20 QC96
44
T=10 Epoch: 45/50 - Loss: 0.0059 b_loss=0.0058 - T: 13.81s/epoch ,tempo_previto=380.79 min0.01 nl20 QC96
45
T=10 Epoch: 46/50 - Loss: 0.0059 b_loss=0.0058 - T: 15.24s/epoch ,tempo_previto=420.15 min0.01 nl20 QC96
46
T=10 Epoch: 47/50 - Loss: 0.0058 b_loss=0.0058 - T: 15.99s/epoch ,tempo_previto=440.45 min0.01 nl20 QC96
47
T=10 Epoch: 48/50 - Loss: 0.0058 b_loss=0.0058 - T: 14.75s/epoch ,tempo_previto=406.05 min0.01 nl20 QC96
48
T=10 Epoch: 49/50 - Loss: 0.0053 b_loss=0.0053 - T: 14.52s/epoch ,tempo_previto=399.52 min0.01 nl20 QC96
49
T=10 Epoch: 50/50 - Loss: 0.0065 b_loss=0.0053 - T: 13.70s/epoch ,tempo_previto=376.84 min0.01 nl20 QC96
(11,)
0
T=10 Epoch: 1/50 - Loss: 0.2845 b_loss=0.2845 - T: 14.67s/epoch ,tempo_previto=342.10 min0.08 nl20 QC96
1
T=10 Epoch: 2/50 - Loss: 0.1504 b_loss=0.1504 - T: 14.86s/epoch ,tempo_previto=346.15 min0.08 nl20 QC96
2
T=10 Epoch: 3/50 - Loss: 0.

T=10 Epoch: 21/50 - Loss: 0.0347 b_loss=0.0305 - T: 16.42s/epoch ,tempo_previto=609.90 min0.1 nl20 QC112
21
T=10 Epoch: 22/50 - Loss: 0.0325 b_loss=0.0305 - T: 14.94s/epoch ,tempo_previto=554.85 min0.1 nl20 QC112
22
T=10 Epoch: 23/50 - Loss: 0.0302 b_loss=0.0302 - T: 14.48s/epoch ,tempo_previto=537.42 min0.1 nl20 QC112
23
T=10 Epoch: 24/50 - Loss: 0.0318 b_loss=0.0302 - T: 16.00s/epoch ,tempo_previto=593.65 min0.1 nl20 QC112
24
T=10 Epoch: 25/50 - Loss: 0.0361 b_loss=0.0302 - T: 15.05s/epoch ,tempo_previto=558.21 min0.1 nl20 QC112
25
T=10 Epoch: 26/50 - Loss: 0.0316 b_loss=0.0302 - T: 14.12s/epoch ,tempo_previto=523.54 min0.1 nl20 QC112
26
T=10 Epoch: 27/50 - Loss: 0.0354 b_loss=0.0302 - T: 15.04s/epoch ,tempo_previto=557.26 min0.1 nl20 QC112
27
T=10 Epoch: 28/50 - Loss: 0.0341 b_loss=0.0302 - T: 13.34s/epoch ,tempo_previto=494.04 min0.1 nl20 QC112
28
T=10 Epoch: 29/50 - Loss: 0.0333 b_loss=0.0302 - T: 14.73s/epoch ,tempo_previto=545.43 min0.1 nl20 QC112
29
T=10 Epoch: 30/50 - Loss: 0.

T=10 Epoch: 47/50 - Loss: 0.0109 b_loss=0.0109 - T: 21.00s/epoch ,tempo_previto=666.11 min0.05 nl20 QC112
47
T=10 Epoch: 48/50 - Loss: 0.0115 b_loss=0.0109 - T: 21.61s/epoch ,tempo_previto=684.89 min0.05 nl20 QC112
48
T=10 Epoch: 49/50 - Loss: 0.0115 b_loss=0.0109 - T: 21.98s/epoch ,tempo_previto=696.53 min0.05 nl20 QC112
49
T=10 Epoch: 50/50 - Loss: 0.0110 b_loss=0.0109 - T: 15.81s/epoch ,tempo_previto=500.74 min0.05 nl20 QC112
(11,)
0
T=10 Epoch: 1/50 - Loss: 0.1978 b_loss=0.1978 - T: 14.32s/epoch ,tempo_previto=393.58 min0.01 nl20 QC112
1
T=10 Epoch: 2/50 - Loss: 0.1061 b_loss=0.1061 - T: 13.96s/epoch ,tempo_previto=383.40 min0.01 nl20 QC112
2
T=10 Epoch: 3/50 - Loss: 0.0757 b_loss=0.0757 - T: 14.84s/epoch ,tempo_previto=407.23 min0.01 nl20 QC112
3
T=10 Epoch: 4/50 - Loss: 0.0512 b_loss=0.0512 - T: 15.64s/epoch ,tempo_previto=429.17 min0.01 nl20 QC112
4
T=10 Epoch: 5/50 - Loss: 0.0366 b_loss=0.0366 - T: 16.92s/epoch ,tempo_previto=463.88 min0.01 nl20 QC112
5
T=10 Epoch: 6/50 - Loss:

T=10 Epoch: 23/50 - Loss: 0.0245 b_loss=0.0213 - T: 13.60s/epoch ,tempo_previto=300.87 min0.08 nl20 QC112
23
T=10 Epoch: 24/50 - Loss: 0.0256 b_loss=0.0213 - T: 13.84s/epoch ,tempo_previto=305.80 min0.08 nl20 QC112
24
T=10 Epoch: 25/50 - Loss: 0.0206 b_loss=0.0206 - T: 14.32s/epoch ,tempo_previto=316.14 min0.08 nl20 QC112
25
T=10 Epoch: 26/50 - Loss: 0.0260 b_loss=0.0206 - T: 14.53s/epoch ,tempo_previto=320.59 min0.08 nl20 QC112
26
T=10 Epoch: 27/50 - Loss: 0.0235 b_loss=0.0206 - T: 14.17s/epoch ,tempo_previto=312.47 min0.08 nl20 QC112
27
T=10 Epoch: 28/50 - Loss: 0.0243 b_loss=0.0206 - T: 12.89s/epoch ,tempo_previto=284.02 min0.08 nl20 QC112
28
T=10 Epoch: 29/50 - Loss: 0.0228 b_loss=0.0206 - T: 13.84s/epoch ,tempo_previto=304.60 min0.08 nl20 QC112
29
T=10 Epoch: 30/50 - Loss: 0.0233 b_loss=0.0206 - T: 13.65s/epoch ,tempo_previto=300.31 min0.08 nl20 QC112
30
T=10 Epoch: 31/50 - Loss: 0.0230 b_loss=0.0206 - T: 14.04s/epoch ,tempo_previto=308.74 min0.08 nl20 QC112
31
T=10 Epoch: 32/50 -

T=10 Epoch: 49/50 - Loss: 0.0311 b_loss=0.0302 - T: 15.37s/epoch ,tempo_previto=551.15 min0.1 nl20 QC120
49
T=10 Epoch: 50/50 - Loss: 0.0301 b_loss=0.0301 - T: 14.37s/epoch ,tempo_previto=514.95 min0.1 nl20 QC120
(11,)
0
T=10 Epoch: 1/50 - Loss: 0.2482 b_loss=0.2482 - T: 14.81s/epoch ,tempo_previto=468.84 min0.05 nl20 QC120
1
T=10 Epoch: 2/50 - Loss: 0.1474 b_loss=0.1474 - T: 14.44s/epoch ,tempo_previto=456.73 min0.05 nl20 QC120
2
T=10 Epoch: 3/50 - Loss: 0.1228 b_loss=0.1228 - T: 14.12s/epoch ,tempo_previto=446.47 min0.05 nl20 QC120
3
T=10 Epoch: 4/50 - Loss: 0.0906 b_loss=0.0906 - T: 13.90s/epoch ,tempo_previto=439.23 min0.05 nl20 QC120
4
T=10 Epoch: 5/50 - Loss: 0.0808 b_loss=0.0808 - T: 13.03s/epoch ,tempo_previto=411.47 min0.05 nl20 QC120
5
T=10 Epoch: 6/50 - Loss: 0.0695 b_loss=0.0695 - T: 14.30s/epoch ,tempo_previto=451.27 min0.05 nl20 QC120
6
T=10 Epoch: 7/50 - Loss: 0.0613 b_loss=0.0613 - T: 15.34s/epoch ,tempo_previto=483.82 min0.05 nl20 QC120
7
T=10 Epoch: 8/50 - Loss: 0.053